### Filtrerer ut datasett med for liten datamengde. Kan justeres i "terskel" variabelen.
Dette er en måte å filtrere ut datasett som inneholder lite til ingen data. Gjør dette fordi vi ønsker å finne ut hvilket datasett som er gunstig å undersøke videre.

In [ ]:
import requests
import pandas as pd

# Definer en terskel for tileCount
terskel = 100000


# Juster Pandas-opsjoner for å vise flere rader og kolonner om ønskelig
pd.set_option('display.max_rows', 374)      # Viser opptil 374 rader
pd.set_option('display.max_columns', None) # Viser alle kolonner
pd.set_option('display.width', None)       # Unngå avkorting av kolonner

# Sett base-URL for API-et
base_url = "https://sealevel-nexus.jpl.nasa.gov"
endpoint = "/list"
url = base_url + endpoint

response = requests.get(url)

if response.status_code == 200:
    # data er en liste med ordbøker (datasetter)
    data = response.json()

    # Konverter data til en DataFrame
    df = pd.DataFrame(data)

    # Filtrer ut rader med tilecount under terskelen
    df_filtrert = df[df["tileCount"] >= terskel]

    # Skriv ut filtrert DataFrame i tabellform uten indeks
    print("Filtrert tabell (tileCount >= {}):".format(terskel))
    print(df_filtrert.to_string(index=False))
else:
    print("Feil ved henting av datasett:", response.status_code)

### Filtrerer ut datasettene som ikke inneholder SSH (Sea Surface Height) og som ikke er over datamengde terskelen. 

In [ ]:
import requests
import pandas as pd

# Definer terskel for tileCount
terskel = 10000

# Juster Pandas-opsjoner for å vise alle kolonner og flere rader i en bred tabell
pd.set_option('display.max_rows', 374)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Sett base-URL og endepunkt for API-et
base_url = "https://sealevel-nexus.jpl.nasa.gov"
endpoint = "/list"
url = base_url + endpoint

response = requests.get(url)

if response.status_code == 200:
    data = response.json()  
    df = pd.DataFrame(data)

    # Filtrer ut datasett med tilecount under terskelen
    df_filtrert = df[df["tileCount"] >= terskel]

    # Velg kun de datasettene som inneholder "SSH" i for kolonnen 'shortName'
    df_ssh = df_filtrert[df_filtrert["shortName"].str.contains("SSH", case=False, na=False)]

    # Skriv ut den filtrerte tabellen uten indeks
    print(f"Datasett med tileCount >= {terskel} og som inneholder 'SSH':")
    print(df_ssh.to_string(index=False))
else:
    print("Feil ved henting av datasett:", response.status_code)

### Finner SSH datasettene i API-en ved bruk av list comprehensions
Dette går gjennom hele datasettet (som er en liste med ordbøker) og lager en ny liste som kun inneholder de datasettene med "SSH" i shortName. Det er en effektiv og lesbar måte å filtrere og hente ut relevante datasett på. 


In [ ]:
import requests
import pandas as pd


# Hent data fra API-et

base_url = "https://sealevel-nexus.jpl.nasa.gov"
endpoint = "/list"
url = base_url + endpoint

response = requests.get(url)
if response.status_code == 200:
    data = response.json()
    df = pd.DataFrame(data)
else:
    print("Feil ved henting av datasett:", response.status_code)
    exit(1)



# 1. Bruker list comprehension for å hente ut SSH-datasett
ssh_datasets = [d["shortName"] for d in data if "SSH" in d.get("shortName", "")]
print("Liste over datasettnavn som inneholder 'SSH':")
print(ssh_datasets)
print("\n-----------------------------------------\n")



### Filtrerer ut datasett som ikke inneholder "SSH" ved bruk av itererasjoner. 
Regner ut differansen på antall år målingene har blitt tatt. Deretter kan vi se videre på egenskapene til de ulike datasettene.

In [ ]:
import requests
import pandas as pd 


# Hent data fra API-et

base_url = "https://sealevel-nexus.jpl.nasa.gov"
endpoint = "/list"
url = base_url + endpoint

response = requests.get(url)
if response.status_code == 200:
    data = response.json()
    df = pd.DataFrame(data)
else:
    print("Feil ved henting av datasett:", response.status_code)
    exit(1)


# Konverter iso_start og iso_end til datetime, og beregn varighet i år
df["iso_start_dt"] = pd.to_datetime(df["iso_start"])
df["iso_end_dt"]   = pd.to_datetime(df["iso_end"])

# Tar differansen i dager og deler på 365.25 for å få en tilnærming i år
df["duration_years"] = (df["iso_end_dt"] - df["iso_start_dt"]).dt.days / 365.25


# 2. Bruker iterasjoner for å skrive ut egenskaper
print("Detaljer om SSH-datasettene (ved iterasjoner):")
for index, row in df.iterrows():
    if "SSH" in row["shortName"]:
        print(f"Index {index}: shortName = {row['shortName']}, "
              f"title = {row.get('title')}, "
              f"tileCount = {row.get('tileCount')}, "
              f"duration_years = {row.get('duration_years'):.2f}")
print("\n-----------------------------------------\n")


### Anlyserer SSH datasettene ved bruk av Pandas SQL
Pandasql lar deg bruke SQL-spørringer direkte på Pandas DataFrames. Dette kan gjøre det enklere og mer intuitivt å jobbe med data for de som er godt kjent med SQL. I stedet for å måtte bruke flere forskjellige Pandas-funksjoner når du skal filtrere, sortere eller gruppere data, kan du gjøre alt med én SQL-setning. Dette gir en mer oversiktlig kode, spesielt når du gjør kompliserte operasjoner. I tillegg gjør Pandasql det lettere å kombinere flere DataFrames med JOIN, noe som ofte er litt vanskeligere med vanlige Pandas-funksjoner.

In [2]:
import requests
import pandas as pd
from pandasql import sqldf  

# API-endepunkt og URL
base_url = "https://sealevel-nexus.jpl.nasa.gov"
endpoint = "/list"
url = base_url + endpoint

# Hent data fra API-et
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    df_datasets = pd.DataFrame(data)
else:
    print(f"Feil ved henting av datasett: {response.status_code} - {response.text}")
    exit(1)

# Konverter datokolonner til datetime-format for videre beregning
df_datasets["iso_start_dt"] = pd.to_datetime(df_datasets["iso_start"])
df_datasets["iso_end_dt"] = pd.to_datetime(df_datasets["iso_end"])

# Funksjon for enklere SQL-spørringer
pysqldf = lambda q: sqldf(q, globals())

# SQL-spørring for å hente ut SSH-datasett og varighet
query = """
    SELECT 
        shortName,
        title,
        tileCount,
        iso_start,
        iso_end,
        (julianday(iso_end_dt) - julianday(iso_start_dt)) / 365.25 AS duration_years
    FROM df_datasets
    WHERE shortName LIKE '%SSH%'
    ORDER BY iso_start_dt ASC;
"""

# Utfør SQL-spørring
df_ssh_sql = pysqldf(query)

# Vis resultatet
print("SSH-datasett fra SQL-spørring:")
print(df_ssh_sql.to_string(index=False))


SSH-datasett fra SQL-spørring:
                               shortName                                    title  tileCount                iso_start                  iso_end  duration_years
NASA_SSH_REF_SIMPLE_GRID_V1_Monthly_clim NASA_SSH_REF_SIMPLE_GRID_V1_Monthly_clim       2604 1970-01-16T00:00:00+0000 1970-12-16T00:00:00+0000        0.914442
      SSHIBC_ECCO_version4_release4_clim       SSHIBC_ECCO_version4_release4_clim       3024 1970-01-16T00:00:00+0000 1970-12-16T00:00:00+0000        0.914442
    SSHNOIBC_ECCO_version4_release4_clim     SSHNOIBC_ECCO_version4_release4_clim       3024 1970-01-16T00:00:00+0000 1970-12-16T00:00:00+0000        0.914442
         SSH_ECCO_version4_release4_clim          SSH_ECCO_version4_release4_clim       3024 1970-01-16T00:00:00+0000 1970-12-16T00:00:00+0000        0.914442
           SSHIBC_ECCO_version4_release4            SSHIBC_ECCO_version4_release4      78624 1992-01-16T18:00:00+0000 2017-12-16T06:00:00+0000       25.915127
         SSHNOI

### Velger å se på "NASA_SSH_REF_SIMPLE_GRID_V1_Monthly" videre.
Ser i tabellen ovenfor at "NASA_SSH_REF_SIMPLE_GRID_V1_Monthly" har største mengden data og målinger over den lengste perioden. Derfor vil det være naturlig å jobbe med dette datasettet videre. 

### Setter opp dataen i "NASA_SSH_REF_SIMPLE_GRID_V1_Monthly" i en tabell.
 Det kan fort oppstå manglende verdier eller målingsfeil i slike datasett. Dette løser vi ved å lage et intervall for realistiske målinger og tester "mean" verdiene opp mot intervallet. Deretter teller vi antall ganger vi møter verdier utenfor intervallet eller vi møter en tom verdi. Slik finner vi antall feilverdier og ser om dataen er klar for videre analyse. 

In [3]:
import requests
import pandas as pd
from datetime import datetime

base_url = "https://sealevel-nexus.jpl.nasa.gov"
endpoint = "/timeSeriesSpark"
url = base_url + endpoint

params = {
    "ds": "NASA_SSH_REF_SIMPLE_GRID_V1_Monthly",
    "b": "-180.0,-90.0,180.0,90.0",
    "startTime": "1992-01-01T00:00:00Z",
    "endTime": "2025-01-01T00:00:00Z",
    "output": "JSON"
}

response = requests.get(url, params=params)

if response.status_code == 200:
    ts_data = response.json()
else:
    print("Feil ved henting av tidsseriedata:", response.status_code, response.text)
    ts_data = None

if ts_data and "data" in ts_data:
    # ts_data["data"] er en liste med mange "sub-lister",
    # hvor hver sub-liste inneholder ordbøker med felter som 'mean' og 'iso_time'.
    data_list_of_lists = ts_data["data"]

    # Slå sammen alle underlister til én flat liste med ordbøker
    all_records = []
    for sublist in data_list_of_lists:
        all_records.extend(sublist)

    print(f"Antall registreringer mottatt: {len(all_records)}")

    # Konverter til Pandas-DataFrame for å få en oversiktlig tabell
    df = pd.DataFrame(all_records)

    # Ser de utvalgte radene i DataFrame
    print("\n--- Utdrag av datasettet (første 386 radene) ---")
    print(df.head(386))

else:
    print("Responsen har ikke den forventede strukturen eller er tom.")

# Definerer det gyldige intervallet for havnivåverdier (kan justeres nedenfor)
min_valid = -0.1
max_valid = 0.1

# Sjekk at 'mean'-kolonnen finnes i DataFrame
if 'mean' in df.columns:
    # Finner manglende (NaN) verdier eller verdier utenfor intervallet
    invalid_mask = df['mean'].isna() | (df['mean'] < min_valid) | (df['mean'] > max_valid)
    
    # Teller antall usaklige verdier
    invalid_count = invalid_mask.sum()
    
    print("\nAntall usaklige verdier i havnivådata:", invalid_count)
else:
    print("Kolonnen 'mean' finnes ikke i datasettet.")




   





Antall registreringer mottatt: 386

--- Utdrag av datasettet (første 386 radene) ---
          min       max      mean     cnt       std        time  \
0   -0.388882  0.431853 -0.000171  145539  0.056767   717897600   
1   -0.366509  0.296866 -0.004225  147346  0.050972   720576000   
2   -0.311839  0.319928 -0.012979  147965  0.050031   723168000   
3   -0.330625  0.358001 -0.018975  147545  0.055296   725846400   
4   -0.362726  0.390943 -0.020470  147413  0.059630   728524800   
..        ...       ...       ...     ...       ...         ...   
381 -0.542817  0.558163  0.079627  146544  0.053102  1719792000   
382 -0.356477  0.744147  0.082227  145662  0.062035  1722470400   
383 -0.362203  0.837300  0.086179  145581  0.064875  1725148800   
384 -0.433927  0.828580  0.092794  145621  0.064465  1727740800   
385 -0.490822  0.806355  0.093230  146428  0.061592  1730419200   

                     iso_time  meanLowPass  maxLowPass  minLowPass  ds  
0    1992-10-01T00:00:00+0000    -0.0